In [4]:
!unzip  DocumentCollection.zip

Archive:  DocumentCollection.zip
   creating: DocumentCollection/
 extracting: DocumentCollection/1.txt  
 extracting: DocumentCollection/10.txt  
 extracting: DocumentCollection/2.txt  
 extracting: DocumentCollection/3.txt  
 extracting: DocumentCollection/4.txt  
 extracting: DocumentCollection/5.txt  
 extracting: DocumentCollection/6.txt  
 extracting: DocumentCollection/7.txt  
 extracting: DocumentCollection/8.txt  
 extracting: DocumentCollection/9.txt  


In [5]:
import os
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from natsort import natsorted

stop_words=set(stopwords.words("english")) # in on at
stop_words.remove('in')
stop_words.remove('to')
stop_words.remove('where')
myFiles= natsorted(os.listdir("/content/DocumentCollection"))
print(myFiles)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['1.txt', '2.txt', '3.txt', '4.txt', '5.txt', '6.txt', '7.txt', '8.txt', '9.txt', '10.txt']


In [6]:
DocTerms=[]
Porter_Stemmer=PorterStemmer()
for printfiles in myFiles:
    with open(f'/content/DocumentCollection/{printfiles}',"r") as fn:
        doc=fn.readline()
        words= word_tokenize(doc)
        terms=[]
        for word in words:
             if word not in stop_words:
                Porter_Stemmer.stem(word)
                terms.append(word)
        DocTerms.append(terms)

print(DocTerms)

[['antony', 'brutus', 'caeser', 'cleopatra', 'mercy', 'worser'], ['antony', 'brutus', 'caeser', 'calpurnia'], ['mercy', 'worser'], ['brutus', 'caeser', 'mercy', 'worser'], ['caeser', 'mercy', 'worser'], ['antony', 'caeser', 'mercy'], ['angels', 'fools', 'fear', 'in', 'rush', 'to', 'tread', 'where'], ['angels', 'fools', 'fear', 'in', 'rush', 'to', 'tread', 'where'], ['angels', 'fools', 'in', 'rush', 'to', 'tread', 'where'], ['fools', 'fear', 'in', 'rush', 'to', 'tread', 'where']]


In [7]:



document_number = 0
positional_index = {}  #dictionary
for document in DocTerms:

    # For position and term in the tokens.
    for positional, term in enumerate(document):
        # print(pos, '-->' ,term)

        # If term already exists in the positional index dictionary.
        if term in positional_index:

            # Increment total freq by 1.
            positional_index[term][0] = positional_index[term][0] + 1

            # Check if the term has existed in that DocID before.
            if document_number in positional_index[term][1]:
                positional_index[term][1][document_number].append(positional)

            else:
                positional_index[term][1][document_number] = [positional]

        # If term does not exist in the positional index dictionary
        # (first encounter).
        else:

            # Initialize the list.
            positional_index[term] = []
            # The total frequency is 1.
            positional_index[term].append(1)
            # The postings list is initially empty.
            positional_index[term].append({})
            # Add doc ID to postings list.
            positional_index[term][1][document_number] = [positional]

    # Increment the file no. counter for document ID mapping
    document_number += 1

print('************************Positional index*************************')
print(positional_index)


************************Positional index*************************
{'antony': [3, {0: [0], 1: [0], 5: [0]}], 'brutus': [3, {0: [1], 1: [1], 3: [0]}], 'caeser': [5, {0: [2], 1: [2], 3: [1], 4: [0], 5: [1]}], 'cleopatra': [1, {0: [3]}], 'mercy': [5, {0: [4], 2: [0], 3: [2], 4: [1], 5: [2]}], 'worser': [4, {0: [5], 2: [1], 3: [3], 4: [2]}], 'calpurnia': [1, {1: [3]}], 'angels': [3, {6: [0], 7: [0], 8: [0]}], 'fools': [4, {6: [1], 7: [1], 8: [1], 9: [0]}], 'fear': [3, {6: [2], 7: [2], 9: [1]}], 'in': [4, {6: [3], 7: [3], 8: [2], 9: [2]}], 'rush': [4, {6: [4], 7: [4], 8: [3], 9: [3]}], 'to': [4, {6: [5], 7: [5], 8: [4], 9: [4]}], 'tread': [4, {6: [6], 7: [6], 8: [5], 9: [5]}], 'where': [4, {6: [7], 7: [7], 8: [6], 9: [6]}]}


In [27]:
query =  "antony brutus"
def query_phrase(query):
 final_list =[[]for i in range(10)] # each array represent doc final_list[1] =>doc1
 for word in query.split(): #word = fools
  if(word not in positional_index ):
    print("not found")
  else:
     for key in positional_index[word][1].keys():  #key 6    keys 6 ,7,8,9 #keys is docID
            if final_list[key]!=[]: #if doc 6 not empty
                if final_list[key][-1]==positional_index[word][1][key][0]-1:  #[-1] last element / if the postion of word  == postion of onther word -1 /عشان احنا عايزين الكلمتان وراه بعض
                    final_list[key].append(positional_index[word][1][key][0])#append postion of word in doc 6
            else:     #doc 6 is ampty
                final_list[key].append(positional_index[word][1][key][0])     #append postion of word in doc 6
 positions=[]
 for pos,list in enumerate(final_list,start=1):

         if len(list)== len(query.split()):
              #print(pos,list)
              positions.append('doc'+str(pos))
 return positions
query_phrase(query)

['doc1', 'doc2']

In [10]:
import pandas as pd
import math
all_words = [] #list contain all terms
for doc in DocTerms:
    for word in doc:
        all_words.append(word)

def get_term_freq(doc):
    words_found = dict.fromkeys(all_words, 0)  #dictionary keys=> term , values=>0
    for word in doc:
        words_found[word] += 1
    return words_found

term_freq = pd.DataFrame(get_term_freq(DocTerms[0]).values(), index=get_term_freq(DocTerms[0]).keys()) #index =>name of terms

for i in range(1, len(DocTerms)):
    term_freq[i] = get_term_freq(DocTerms[i]).values()

term_freq.columns = ['doc'+str(i) for i in range(1, 11)]
print("*****************************term_freq*****************************")
print(term_freq)

*****************************term_freq*****************************
           doc1  doc2  doc3  doc4  doc5  doc6  doc7  doc8  doc9  doc10
antony        1     1     0     0     0     1     0     0     0      0
brutus        1     1     0     1     0     0     0     0     0      0
caeser        1     1     0     1     1     1     0     0     0      0
cleopatra     1     0     0     0     0     0     0     0     0      0
mercy         1     0     1     1     1     1     0     0     0      0
worser        1     0     1     1     1     0     0     0     0      0
calpurnia     0     1     0     0     0     0     0     0     0      0
angels        0     0     0     0     0     0     1     1     1      0
fools         0     0     0     0     0     0     1     1     1      1
fear          0     0     0     0     0     0     1     1     0      1
in            0     0     0     0     0     0     1     1     1      1
rush          0     0     0     0     0     0     1     1     1      1
to       

In [11]:
def get_weighted_term_freq(x):
    if x > 0:
        return math.log10(x) + 1
    return 0

for i in range(1, len(DocTerms)+1):
    term_freq['doc'+str(i)] = term_freq['doc'+str(i)].apply(get_weighted_term_freq)

print(term_freq)

           doc1  doc2  doc3  doc4  doc5  doc6  doc7  doc8  doc9  doc10
antony      1.0   1.0   0.0   0.0   0.0   1.0   0.0   0.0   0.0    0.0
brutus      1.0   1.0   0.0   1.0   0.0   0.0   0.0   0.0   0.0    0.0
caeser      1.0   1.0   0.0   1.0   1.0   1.0   0.0   0.0   0.0    0.0
cleopatra   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0
mercy       1.0   0.0   1.0   1.0   1.0   1.0   0.0   0.0   0.0    0.0
worser      1.0   0.0   1.0   1.0   1.0   0.0   0.0   0.0   0.0    0.0
calpurnia   0.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0
angels      0.0   0.0   0.0   0.0   0.0   0.0   1.0   1.0   1.0    0.0
fools       0.0   0.0   0.0   0.0   0.0   0.0   1.0   1.0   1.0    1.0
fear        0.0   0.0   0.0   0.0   0.0   0.0   1.0   1.0   0.0    1.0
in          0.0   0.0   0.0   0.0   0.0   0.0   1.0   1.0   1.0    1.0
rush        0.0   0.0   0.0   0.0   0.0   0.0   1.0   1.0   1.0    1.0
to          0.0   0.0   0.0   0.0   0.0   0.0   1.0   1.0   1.0    1.0
tread 

In [12]:
tfd = pd.DataFrame(columns=['freq', 'idf'])

for i in range(len(term_freq)):

    frequency = term_freq.iloc[i].values.sum()  #iloc return location  iloc[0]=>return row 0
    tfd.loc[i, 'freq'] = frequency #insert position and column

    tfd.loc[i, 'idf'] = math.log10(10 / (float(frequency)))

tfd.index = term_freq.index

print(tfd)

term_freq_inve_doc_freq = term_freq.multiply(tfd['idf'], axis=0)
print(term_freq_inve_doc_freq)

          freq       idf
antony     3.0  0.522879
brutus     3.0  0.522879
caeser     5.0   0.30103
cleopatra  1.0       1.0
mercy      5.0   0.30103
worser     4.0   0.39794
calpurnia  1.0       1.0
angels     3.0  0.522879
fools      4.0   0.39794
fear       3.0  0.522879
in         4.0   0.39794
rush       4.0   0.39794
to         4.0   0.39794
tread      4.0   0.39794
where      4.0   0.39794
               doc1      doc2     doc3      doc4     doc5      doc6      doc7  \
antony     0.522879  0.522879      0.0       0.0      0.0  0.522879       0.0   
brutus     0.522879  0.522879      0.0  0.522879      0.0       0.0       0.0   
caeser      0.30103   0.30103      0.0   0.30103  0.30103   0.30103       0.0   
cleopatra       1.0       0.0      0.0       0.0      0.0       0.0       0.0   
mercy       0.30103       0.0  0.30103   0.30103  0.30103   0.30103       0.0   
worser      0.39794       0.0  0.39794   0.39794  0.39794       0.0       0.0   
calpurnia       0.0       1.0    

In [13]:
import numpy as np
document_length = pd.DataFrame()
def get_docs_length(col):
    return np.sqrt(term_freq_inve_doc_freq[col].apply(lambda x: x**2).sum())

for column in term_freq_inve_doc_freq.columns:
    document_length.loc[0, column+'_len'] = get_docs_length(column)
print(document_length)

   doc1_len  doc2_len  doc3_len  doc4_len  doc5_len  doc6_len  doc7_len  \
0  1.373462  1.279618  0.498974  0.782941  0.582747   0.67427  1.223496   

   doc8_len  doc9_len  doc10_len  
0  1.223496  1.106137   1.106137  


In [14]:
normalized_term_freq_idf = pd.DataFrame()

def get_normalized(col, x):
    try:
        return x / document_length[col+'_len'].values[0]
    except:
        return 0

for column in term_freq_inve_doc_freq.columns:
    normalized_term_freq_idf[column] = term_freq_inve_doc_freq[column].apply(lambda x : get_normalized(column, x))


normalized_term_freq_idf

,doc1,doc2,doc3,doc4,doc5,doc6,doc7,doc8,doc9,doc10
antony,0.380701,0.408621,0.000000,0.000000,0.000000,0.775474,0.000000,0.000000,0.000000,0.000000
brutus,0.380701,0.408621,0.000000,0.667839,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
caeser,0.219176,0.235250,0.000000,0.384486,0.516570,0.446453,0.000000,0.000000,0.000000,0.000000
cleopatra,0.728087,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
mercy,0.219176,0.000000,0.603298,0.384486,0.516570,0.446453,0.000000,0.000000,0.000000,0.000000
worser,0.289735,0.000000,0.797516,0.508263,0.682869,0.000000,0.000000,0.000000,0.000000,0.000000
calpurnia,0.000000,0.781483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
angels,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.427365,0.427365,0.472707,0.000000
fools,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.325248,0.325248,0.359756,0.359756
fear,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.427365,0.427365,0.000000,0.472707


In [52]:
q="antony brutus"
def returned(q):
 document_found=query_phrase(q)
 if document_found==[]:
  return  'not found'
 query = pd.DataFrame(index=normalized_term_freq_idf.index)
 index_list = []
 for i in range(len(normalized_term_freq_idf.index)):
   index_list.append(normalized_term_freq_idf.index[i])
 query['tf'] = [ 1 if x in q.split() else 0 for x in  index_list]
 query['w_tf'] = query['tf'].apply(lambda x :get_weighted_term_freq(x))
 product=normalized_term_freq_idf.multiply(query['w_tf'],axis=0)
 query['idf']=tfd['idf']*query['w_tf']
 query['tf_idf']=query['w_tf']*query['idf']
 query['norm'] =0
 for i in range(len(query)):
     query['norm'].iloc[i]=float(query['idf'].iloc[i])/math.sqrt(sum(query['idf'].values**2))
 product2= product.multiply(query['norm'],axis=0)
 query_length=math.sqrt(sum([x**2 for x in query['idf'].loc[q.split()]]))
 scores={}
 for col in document_found:
    scores[col]= product2[col].sum()
 names_docs=[]
 for key in scores.keys():
   names_docs.append(key)
 prod_res= product2[names_docs].loc[q.split()]
 print(query)
 print("\n product (query * matched docs)")
 print(prod_res)
 print("\n sum of product")
 print(prod_res.sum())
 print("\n query length")
 print( query_length)
 print("\n similarity")
 print(scores)
 print("\n returned docs")
 ranked =sorted(scores.items(),key=lambda x : x[1], reverse=True)
 for doc in ranked:
    print(doc[0],end=' ')
 print("\n")
returned(q)

           tf  w_tf       idf    tf_idf      norm
antony      1   1.0  0.522879  0.522879  0.707107
brutus      1   1.0  0.522879  0.522879  0.707107
caeser      0   0.0       0.0       0.0  0.000000
cleopatra   0   0.0       0.0       0.0  0.000000
mercy       0   0.0       0.0       0.0  0.000000
worser      0   0.0       0.0       0.0  0.000000
calpurnia   0   0.0       0.0       0.0  0.000000
angels      0   0.0       0.0       0.0  0.000000
fools       0   0.0       0.0       0.0  0.000000
fear        0   0.0       0.0       0.0  0.000000
in          0   0.0       0.0       0.0  0.000000
rush        0   0.0       0.0       0.0  0.000000
to          0   0.0       0.0       0.0  0.000000
tread       0   0.0       0.0       0.0  0.000000
where       0   0.0       0.0       0.0  0.000000

 product (query * matched docs)
            doc1      doc2
antony  0.269196  0.288939
brutus  0.269196  0.288939

 sum of product
doc1    0.538393
doc2    0.577877
dtype: float64

 query length
0.739

<ipython-input-52-d7a74d694400>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query['norm'].iloc[i]=float(query['idf'].iloc[i])/math.sqrt(sum(query['idf'].values**2))
